In [67]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib
import seaborn as sns
import funciones as fn

import openpyxl

In [68]:
def preprocesar():

    #### conectar_base_de_Datos#################
    conn=sql.connect('Data/movies2.db')
    cur=conn.cursor()

    ######## convertir datos crudos a bases filtradas por usuarios que tengan cierto número de calificaciones
    fn.ejecutar_sql('preprocesamiento_n.sql', cur)

    ##### llevar datos que cambian constantemente a python ######
    movies=pd.read_sql('select * from movies_final', conn )
    ratings=pd.read_sql('select * from f_ratings', conn)
    usuarios=pd.read_sql('select distinct (user_id) as user_id from f_ratings',conn)

    #### transformación de datos crudos - Preprocesamiento ################
    
    # Convertir la columna 'year' a tipo entero
    ratings['movie_year'] = ratings['movie_year'].astype('int')
    ratings['movie_year']=ratings.movie_year.astype('int')
    
    ##### escalar para que año esté en el mismo rango ###
    sc=MinMaxScaler()
    ratings[["year_sc"]]=sc.fit_transform(ratings[['movie_year']])
    ratings_dum1=ratings.drop(columns=['movie_id', 'movie_title', 'movie_genres', 'clean_title', 'fecha_nueva'])

    return ratings_dum1,ratings, conn, cur
     

In [69]:
#########################################################################
############## Función para entrenar modelo por cada usuario #############
############## Basado en contenido todo lo visto por el usuario Knn ######
def recomendar(user_id):
    
    ratings_dum1, ratings, conn, cur= preprocesar()
    
    ratings=pd.read_sql('select *from f_reatings where user_id=:user',conn, params={'user':user_id})
    l_ratings=ratings['movie_id'].to_numpy()
    ratings_dum1[['movie_id','clean_title']]=ratings[['movie_id','clean_title']]
    ratings_r=ratings_dum1[ratings_dum1['movie_id'].isin(l_ratings)]
    ratings_r=ratings_r.drop(columns=['movie_id','clean_title'])
    ratings_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=ratings_r.groupby("indice").mean()
    
    
    ratings_nr=ratings_dum1[~ratings_dum1['movie_id'].isin(l_ratings)]
    ratings_nr=ratings_nr.drop(columns=['movie_id','clean_title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(ratings_nr)
    dist, idlist = model.kneighbors(centroide)
    
    ids=idlist[0]
    recomend_b=ratings.loc[ids][['movie_id','clean_title']]
    
    
    return recomend_b

In [70]:

def recomendar(user_id):
    ratings_dum1, ratings, conn, cur = preprocesar()

    # Seleccionar solo los ratings del usuario seleccionado
    ratings = pd.read_sql('select * from f_ratings where user_id=:user', conn, params={'user': user_id})
    l_ratings = ratings['movie_id'].to_numpy()

    # Agregar la columna de movieID de la película a dummies para filtrar y mostrar nombre
    ratings_dum1[['movie_id', 'clean_title']] = ratings[['movie_id', 'clean_title']]

    # Filtrar películas calificadas por el usuario
    ratings_r = ratings_dum1[ratings_dum1['movie_id'].isin(l_ratings)]
    ratings_r = ratings_r.drop(columns=['movie_id', 'clean_title'])
    ratings_r['indice'] = 1  # Para usar group by y que quede en formato pandas tabla de centroide
    centroide = ratings_r.groupby('indice').mean()

    # Filtrar películas no vistas
    ratings_nr = ratings_dum1[~ratings_dum1['movie_id'].isin(l_ratings)]
    ratings_nr = ratings_nr.drop(columns=['movie_id', 'clean_title'])

    # Entrenar modelo
    model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(ratings_nr)
    dist, idlist = model.kneighbors(centroide)

    ids = idlist[0]
    recomend_b = ratings.loc[ids][['clean_title', 'movie_id']]

    return recomend_b


In [71]:
##### Generar recomendaciones para usuario lista de usuarios ####
##### No se hace para todos porque es muy pesado #############
def main(list_user):
    
    recomendaciones_todos=pd.DataFrame()
    for user_id in list_user:
            
        recomendaciones=recomendar(user_id)
        recomendaciones["user_id"]=user_id
        recomendaciones.reset_index(inplace=True,drop=True)
        
        recomendaciones_todos=pd.concat([recomendaciones_todos, recomendaciones])

    recomendaciones_todos.to_excel('G:\\Mi unidad\\AA\\Analitica-en-Marketing\\salidas\\recomendaciones.xlsx')
    recomendaciones_todos.to_csv('G:\\Mi unidad\\AA\\Analitica-en-Marketing\\salidas\\recomendaciones.csv')


if __name__=="__main__":
    list_user=[53,194,275,500 ]
    main(list_user)
    

import sys
sys.executable

ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required by NearestNeighbors.